**@author Güise Rodríguez**

**Execution date: 14/04/2019 21:00 **

# Instalo las librerías necesarias

In [1]:
!pip3 install requests
!pip3 install beautifulsoup4


# Importo las librerías necesarias

In [0]:
import requests, sys, re, csv, urllib.robotparser
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

# It's show time!

## Preparación para el Scraping

### Modifico el user-agent para evitar bloqueos


In [0]:
headers = {
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,\ */*;q=0.8",
"Accept-Encoding": "gzip, deflate, sdch, br",
"Accept-Language": "en-US,en;q=0.8",
"Cache-Control": "no-cache", "dnt": "1",
"Pragma": "no-cache",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/5\ 37.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36" }

### Verifico el  fichero robots.txt

Para verificar que el fichero robots.txt no evita que podamos consultar las páginas con noticias que nos interesan, creamos una instancia del objeto de Python **RobotFileParser**, que con cuya función **can_fetch()** nos permite conocer devolviendo un booleano si este fichero indica o no si se puede rastrear dicho directorio.

In [4]:
rp = urllib.robotparser.RobotFileParser()
rp.set_url("https://www.eldiario.es/robots.txt")
rp.read()
rp.can_fetch("*", "https://www.eldiario.es/focos/")

True

In [5]:
request = requests.get("https://www.eldiario.es/robots.txt", headers=headers)
if request.status_code != 200:
  sys.exit("Error Code: " + str(request.status_code))
soup = BeautifulSoup(request.content)
print(soup.find('p').prettify())

<p>
 User-agent: *
Disallow: /bbtcaptcha/
Disallow: /bbtcomment/create/
Disallow: /bbtcomment/vote/
Disallow: /bbtcomment/report/
Disallow: /bbtcomment/entityComments/
Disallow: /bbtcontent/poll/vote/
Disallow: /bbtcontent/poll/getresults/
Disallow: /props/
Disallow: https://www.eldiario.es/economia/Grupo-Asfi-apropiacion-indebida-blanqueo_0_73143178.html
Disallow: https://www.eldiario.es/norte/navarra/ultima_hora/Condenado-prision-Inaki-Gil-Asfi_0_363614222.html
Disallow: https://www.eldiario.es/norte/navarra/ultima_hora/vale-millon-euros-carcel-llevo_0_353815044.html
Disallow: https://www.eldiario.es/economia/juezas-pagadas-Indra-sentencias-empresa_0_359764337.html
Disallow: https://www.eldiario.es/canariasahora/topsecret/Mckenzie-ataca-nuevo_6_97350295.html
Disallow: /bbtmail/sendEntity/
Disallow: /bbtshop/
Disallow: /bbtstats/
Disallow: /logout.html

Sitemap: https://www.eldiario.es/sitemap_index.xml
Sitemap: https://www.eldiario.es/sitemap_google_news.xml
</p>


También realizo la comprobación imprimiendo los datos del fichero robots.txt para comprobar nuevamente que la página permite rastrear el directorio seleccionado.

Observamos cómo entre los directorios prohibidos a todos los robots no se encuentra **/focos**.

## Código Principal




In [0]:
# Defino las variables que permitirán extraer la información de 5 secciones del periódico digital junto a su profundidad máxima.
base = "https://www.eldiario.es/focos/"
focos = ['mejores_ciudades','vida_digital','creacion_cultural','medio_ambiente','maltrato_animal']
depth = [12,23,41,37,10]
header = ['headline','type','date','section','author','authorInfo','location','url']
csvData = []

In [0]:
def get_author(extract):
  """
  Función que busca todas las etiquetas 'a' que contienen los nombres de los 
  autores/as del contenido y las concatena separadas por comas.
  En caso de no haber información en este campo, se adquiere la información del
  primer contenido del extracto HTML, limpiándose los saltos de línea, guiones y 
  varios espacios seguidos que suelen tener los nombres de colectivos autores.
  """
  authors = []
  for autora in extract.find_all("a"):
    authors.append(re.sub(' +',' ',autora['title']))
  if authors:
    return ",".join(map(str,authors))
  #Si es un colectivo, el nombre aparecería con dicho formato
  return re.sub(' +',' ',re.sub('-','',re.sub('\n','',extract.contents[0]))).strip()

In [0]:
def get_data(extract):
  """
  Función que busca la clase asociada a los artículos de opinión
  en las clases del código HTML y determina así si la información
  contenida en la etiqueta 'span' con clase 'location' (si existiera)
  es información extra sobre el autor o la localización de la noticia. 
  """
  extra = extract.find("span",{"class": "location"})
  if 'lst-item-opinion' in extract.get("class"):
    type = "opinion"
    if extra:
      authorInfo = extra.getText()
      location = ""
  else:
    type = "news"
    if extra:
      authorInfo = ""
      location = extra.getText()
  if extra is None:
    authorInfo = ""
    location = ""
  return type, authorInfo, location

In [0]:
def save_dataset(csvData, header):
  """
  Función que convierte la lista de listas csvData a Dataframe y la guarda en un csv
  utilizando las funciones de la librería Pandas.
  """
  dfObj = pd.DataFrame(csvData, columns= header)
  dfObj.to_csv("eldiario_news.csv",sep=";",index=False,quoting=csv.QUOTE_MINIMAL)

In [0]:
def get_page(i, foco):
  """
  Función que recorre todas las páginas que tiene la sección 'foco',
  descarga la página comprobando posibles errores en la solicitud y
  pasa a la función 'get_noticia()' los datos contenidos en la etiqueta
  'div' con clase 'pg-body', que contiene las noticias del medio.
  """
  for nPage in np.arange(depth[i])+1:
    target = base + foco + "/?page=" + str(nPage) 
    request = requests.get(target, headers=headers)
    if request.status_code != 200:
      sys.exit("Error Code: " + str(request.status_code))
    soup = BeautifulSoup(request.content)
    body = soup.find("div",{"class": "pg-body"})
    get_noticia(body) 

In [0]:
def get_noticia(body):
  """
  Función que recorre las diferentes noticias, buscando las diferentes
  etiquetas 'li' con clase 'lst-item' en las que están contenidas, 
  y obtiene datos como el tipo de noticia y la información opcional
  como información extra del autor o la localización de la noticia llamando
  a la función 'get_data()', el título extrayendo el atributo 'title' de la
  etiqueta 'a', la fecha de publicación de la etiqueta 'span' con clase
  'date', la url relativa del atributo 'href' de la etiqueta 'a' y el autor
  o autores llamando a la función 'get_author()'. Por último, concatena a la lista
  csvData una lista con todos los datos de cada noticia.
  """
  for noticia in body.find_all("li",{"class": "lst-item"}):
        type, authorInfo, location = get_data(noticia)
        datos = noticia.find("span",{"class": "byline"})
        headline = re.sub("\\\\'","\'",noticia.find("a")['title']) #Sustituyo "\\'" por una comilla simple
        date = datos.find("span",{"class": "date"}).getText()
        author = get_author(datos)
        url = "eldiario.es" + noticia.find("a")['href']
        csvData.append([headline,type,date,foco,author,authorInfo,location,url])

In [0]:
"""
Código principal en el que se iteran las diferentes secciones
'focos' y se le pasa junto a su identificador numérico para
que la función 'get_page()' itere todas sus páginas.
Posteriormente llama a la función 'save_dataset' para que 
guarde todas las noticias en un fichero csv.
"""
for i, foco in enumerate(focos):
    get_page(i, foco)
save_dataset(csvData, header)